In [3]:
import pandas as pd
df=pd.read_excel("normal_form.xlsx")

In [4]:
print(df)

    Student Number   Student Name    Exam Score   Support   Date of Birth    \
0             1001.0      Bob Baker          78.0        No      2001-08-25   
1                NaN            NaN           NaN       NaN             NaT   
2                NaN            NaN           NaN       NaN             NaT   
3             1002.0   Sally Davies          55.0       Yes      1999-10-02   
4                NaN            NaN           NaN       NaN             NaT   
5                NaN            NaN           NaN       NaN             NaT   
6             1003.0   Mark Hanmill          90.0        No      1995-06-05   
7                NaN            NaN           NaN       NaN             NaT   
8                NaN            NaN           NaN       NaN             NaT   
9             1004.0       Anas Ali          70.0        No      1980-08-03   
10               NaN            NaN           NaN       NaN             NaT   
11               NaN            NaN           NaN   

In [11]:
df_filled = df.fillna(method='ffill')


print(df_filled)


    Student Number   Student Name    Exam Score   Support   Date of Birth    \
0             1001.0      Bob Baker          78.0        No      2001-08-25   
1             1001.0      Bob Baker          78.0        No      2001-08-25   
2             1001.0      Bob Baker          78.0        No      2001-08-25   
3             1002.0   Sally Davies          55.0       Yes      1999-10-02   
4             1002.0   Sally Davies          55.0       Yes      1999-10-02   
5             1002.0   Sally Davies          55.0       Yes      1999-10-02   
6             1003.0   Mark Hanmill          90.0        No      1995-06-05   
7             1003.0   Mark Hanmill          90.0        No      1995-06-05   
8             1003.0   Mark Hanmill          90.0        No      1995-06-05   
9             1004.0       Anas Ali          70.0        No      1980-08-03   
10            1004.0       Anas Ali          70.0        No      1980-08-03   
11            1004.0       Anas Ali          70.0   

In [14]:
column_names = df_filled.columns
column_names

df_filled.columns = df_filled.columns.str.strip()


cleaned_column_names = df_filled.columns
cleaned_column_names


Index(['Student Number', 'Student Name', 'Exam Score', 'Support',
       'Date of Birth', 'Course Name', 'Exam Boards', 'Teacher Name'],
      dtype='object')

In [15]:

students = df_filled[['Student Number', 'Student Name', 'Support', 'Date of Birth']].drop_duplicates()


courses = df_filled[['Course Name', 'Exam Boards', 'Teacher Name']].drop_duplicates()


scores = df_filled[['Student Number', 'Course Name', 'Exam Score']]

students.head(), courses.head(), scores.head()


(    Student Number  Student Name Support Date of Birth
 0           1001.0     Bob Baker      No    2001-08-25
 3           1002.0  Sally Davies     Yes    1999-10-02
 6           1003.0  Mark Hanmill      No    1995-06-05
 9           1004.0      Anas Ali      No    1980-08-03
 12          1005.0     Cheuk Yin     Yes    2002-05-01,
         Course Name Exam Boards Teacher Name
 0  Computer Science         BCS     Mr Jones
 1             Maths     EdExcel    Ms Parker
 2           Physics         OCR    Mr Peters
 3             Maths         AQA    Ms Parker
 4           Biology        WJEC    Mrs Patel,
    Student Number       Course Name  Exam Score
 0          1001.0  Computer Science        78.0
 1          1001.0             Maths        78.0
 2          1001.0           Physics        78.0
 3          1002.0             Maths        55.0
 4          1002.0           Biology        55.0)

In [16]:
teacher_course = courses[['Course Name', 'Teacher Name']].drop_duplicates()


courses = courses[['Course Name', 'Exam Boards']]

teacher_course.head(), courses.head()


(        Course Name Teacher Name
 0  Computer Science     Mr Jones
 1             Maths    Ms Parker
 2           Physics    Mr Peters
 4           Biology    Mrs Patel
 5             Music   Ms Daniels,
         Course Name Exam Boards
 0  Computer Science         BCS
 1             Maths     EdExcel
 2           Physics         OCR
 3             Maths         AQA
 4           Biology        WJEC)

In [25]:
df_filled = df_filled.rename(columns={
    'Student Number': 'StudentNumber',
    'Student Name': 'StudentName',
    'Course Name': 'CourseName',
    'Exam Score': 'ExamScore',
    'Date of Birth': 'DateOfBirth',
    'Exam Boards': 'ExamBoards',
    'Teacher Name': 'TeacherName'
})

students = df_filled[['StudentNumber', 'StudentName', 'Support', 'DateOfBirth']].drop_duplicates()
courses = df_filled[['CourseName', 'ExamBoards']].drop_duplicates()
scores = df_filled[['StudentNumber', 'CourseName', 'ExamScore']]
teacher_course = df_filled[['CourseName', 'TeacherName']].drop_duplicates()


In [26]:
import sqlite3


# Create SQLite database in memory
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Enable foreign key constraints for SQLite database
cursor.execute("PRAGMA foreign_keys = ON")

# Create tables
cursor.execute('''
CREATE TABLE Students (
    StudentNumber INTEGER PRIMARY KEY,
    StudentName TEXT NOT NULL,
    Support TEXT NOT NULL,
    DateOfBirth DATE NOT NULL
)
''')

cursor.execute('''
CREATE TABLE Courses (
    CourseName TEXT PRIMARY KEY,
    ExamBoards TEXT NOT NULL
)
''')

cursor.execute('''
CREATE TABLE Scores (
    StudentNumber INTEGER,
    CourseName TEXT,
    ExamScore INTEGER,
    PRIMARY KEY (StudentNumber, CourseName),
    FOREIGN KEY (StudentNumber) REFERENCES Students(StudentNumber),
    FOREIGN KEY (CourseName) REFERENCES Courses(CourseName)
)
''')

cursor.execute('''
CREATE TABLE TeacherCourse (
    CourseName TEXT,
    TeacherName TEXT,
    PRIMARY KEY (CourseName, TeacherName),
    FOREIGN KEY (CourseName) REFERENCES Courses(CourseName)
)
''')

# Insert data into tables
students.to_sql('Students', conn, if_exists='append', index=False)
courses.to_sql('Courses', conn, if_exists='append', index=False)
scores.to_sql('Scores', conn, if_exists='append', index=False)
teacher_course.to_sql('TeacherCourse', conn, if_exists='append', index=False)

# Function to test referential integrity
def test_referential_integrity():
    try:
        cursor.execute("INSERT INTO Scores (StudentNumber, CourseName, ExamScore) VALUES (?, ?, ?)", (9999, 'Maths', 80))
        return "Inserted record with non-existent StudentNumber!"
    except sqlite3.IntegrityError:
        try:
            cursor.execute("INSERT INTO Scores (StudentNumber, CourseName, ExamScore) VALUES (?, ?, ?)", (1001, 'NonExistentCourse', 80))
            return "Inserted record with non-existent CourseName!"
        except sqlite3.IntegrityError:
            try:
                cursor.execute("DELETE FROM Students WHERE StudentNumber = ?", (1001,))
                return "Deleted record from Students table that is referenced in Scores table!"
            except sqlite3.IntegrityError:
                return "Referential integrity tests passed!"

# Run the tests
test_referential_integrity()


IntegrityError: UNIQUE constraint failed: Courses.CourseName